In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧑‍🍳 PROMPTCHEF",
    "",
    "Este Jupyter Notebook te permite generar recetas personalizadas y sus imágenes, ya sea a partir de los **ingredientes que tienes en tu heladera** o del **nombre de un plato** específico. Utiliza los modelos de inteligencia artificial de Google Gemini para la generación de texto (recetas) y la generación de imágenes.",
    "",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📦 1. Importaciones Necesarias",
    "",
    "Aquí cargamos todas las librerías que necesitaremos para que nuestra aplicación funcione. Principalmente, `google.genai` para interactuar con Gemini, `PIL` (Pillow) para el manejo de imágenes y `io` para trabajar con datos binarios."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from google import genai\n",
    "from google.genai import types\n",
    "from PIL import Image\n",
    "from io import BytesIO"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔑 2. Configuración del Cliente Gemini",
    "",
    "**¡MUY IMPORTANTE!** Para que la aplicación funcione, debes pegar tu `API Key` de Google Gemini en la línea de código de abajo. Si no tienes una, puedes obtenerla en [Google AI Studio](https://aistudio.google.com/app/apikey).",
    "",
    "**No compartas tu API Key públicamente.**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "client = genai.Client(api_key=\"TU_API_KEY_AQUÍ\") # <-- ¡Pega tu API Key aquí!"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## 👨‍🍳 3. Funciones Principales para Generar Recetas",
    "",
    "A continuación, definimos las funciones que interactuarán con el modelo de lenguaje de Gemini para generar las recetas de acuerdo a tus necesidades."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 🍕 Generar Receta a partir de Ingredientes",
    "",
    "Esta función toma una lista de ingredientes que le proporcionas y genera una receta completa. También puedes especificar preferencias dietéticas o el nivel de dificultad."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generar_receta_por_ingredientes(ingredientes: str, preferencias: str = \"\", dificultad: str = \"intermedia\") -> tuple[str, str]:\n",
    "    \"\"\"\n",
    "    Genera una receta de cocina basada en una lista de ingredientes,\n",
    "    preferencias dietéticas y nivel de dificultad.\n",
    "\n",
    "    Args:\n",
    "        ingredientes (str): Una cadena de texto con los ingredientes disponibles (ej. \"papa, morrón, huevo\").\n",
    "        preferencias (str, optional): Preferencias dietéticas (ej. \"vegetariana\", \"sin gluten\"). Por defecto, \"\".\n",
    "        dificultad (str, optional): Nivel de dificultad (\"fácil\", \"intermedia\", \"avanzada\"). Por defecto, \"intermedia\".\n",
    "\n",
    "    Returns:\n",
    "        tuple[str, str]: Una tupla que contiene la receta generada (str) y el título de la receta (str).\n",
    "                         Retorna un mensaje de error y \"Error\" como título si falla.\n",
    "    \"\"\"\n",
    "    prompt = f\"\"\"\n",
    "    Sos un chef profesional especializado en recetas caseras y saludables.\n",
    "    Con los siguientes ingredientes:\n",
    "    {ingredientes}\n",
    "    Generá una receta que sea {preferencias if preferencias else \"sin restricciones dietarias\"},\n",
    "    de dificultad {dificultad}, pensada para una comida rica y práctica.\n",
    "    Escribí un título, una breve descripción, los ingredientes detallados y los pasos de preparación. Usá un lenguaje claro y simple.\n",
    "    \"\"\"\n",
    "    try:\n",
    "        respuesta = client.models.generate_content(\n",
    "            model=\"gemini-1.5-flash\",\n",
    "            contents=prompt\n",
    "        )\n",
    "        receta = respuesta.text\n",
    "        # Extrae el título de la primera línea de la receta\n",
    "        titulo = receta.strip().split('\\n')[0]\n",
    "        return receta, titulo\n",
    "    except Exception as e:\n",
    "        print(f\"Error al generar la receta por ingredientes: {e}\")\n",
    "        return \"No se pudo generar la receta por ingredientes.\", \"Error\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 🍝 Generar Receta a partir del Nombre de un Plato",
    "",
    "Si ya tienes una idea de qué quieres cocinar, esta función te ayuda a obtener una receta completa para ese plato, incluyendo ingredientes y pasos detallados."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generar_receta_por_plato(nombre_plato: str, preferencias: str = \"\", dificultad: str = \"intermedia\") -> tuple[str, str]:\n",
    "    \"\"\"\n",
    "    Genera una receta de cocina completa para un plato específico,\n",
    "    considerando preferencias dietéticas y nivel de dificultad.\n",
    "\n",
    "    Args:\n",
    "        nombre_plato (str): El nombre del plato deseado (ej. \"Lasagna de Carne\").\n",
    "        preferencias (str, optional): Preferencias dietéticas (ej. \"vegana\", \"baja en carbohidratos\"). Por defecto, \"\".\n",
    "        dificultad (str, optional): Nivel de dificultad (\"fácil\", \"intermedia\", \"avanzada\"). Por defecto, \"intermedia\".\n",
    "\n",
    "    Returns:\n",
    "        tuple[str, str]: Una tupla que contiene la receta generada (str) y el título de la receta (str).\n",
    "                         Retorna un mensaje de error y \"Error\" como título si falla.\n",
    "    \"\"\"\n",
    "    prompt = f\"\"\"\n",
    "    Eres un chef profesional especializado en crear recetas detalladas y caseras.\n",
    "    Dado el nombre del plato: \"{nombre_plato}\".\n",
    "    Por favor, genera una receta completa que incluya:\n",
    "    1. Un título claro y atractivo para el plato.\n",
    "    2. Una breve descripción del plato.\n",
    "    3. Una lista detallada de **todos los ingredientes necesarios**, con cantidades precisas.\n",
    "    4. Los pasos de preparación numerados, claros y fáciles de seguir.\n",
    "    Considera las siguientes especificaciones:\n",
    "    - Preferencias dietéticas: {preferencias if preferencias else \"ninguna\"}\n",
    "    - Dificultad de la receta: {dificultad}\n",
    "    - Usa un lenguaje claro y conciso, como si fuera para un libro de cocina.\n",
    "    \"\"\"\n",
    "    try:\n",
    "        respuesta = client.models.generate_content(\n",
    "            model=\"gemini-1.5-flash\",\n",
    "            contents=prompt\n",
    "        )\n",
    "        receta = respuesta.text\n",
    "        # Extrae el título de la primera línea de la receta\n",
    "        titulo = receta.strip().split('\\n')[0]\n",
    "        return receta, titulo\n",
    "    except Exception as e:\n",
    "        print(f\"Error al generar la receta para '{nombre_plato}': {e}\")\n",
    "        return \"No se pudo generar la receta para el plato.\", \"Error\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## 📸 4. Función para Generar la Imagen del Plato",
    "",
    "Esta función utiliza un modelo de Gemini especializado en generación de imágenes para crear una representación visual del plato a partir de su título. La imagen se guardará en el mismo directorio donde ejecutes este notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generar_imagen_desde_titulo(titulo_receta: str):\n",
    "    \"\"\"\n",
    "    Genera y guarda una imagen de un plato basada en el título de la receta.\n",
    "    La imagen se guarda como un archivo PNG en el mismo directorio.\n",
    "\n",
    "    Args:\n",
    "        titulo_receta (str): El título de la receta para usar como base de la imagen.\n",
    "    \"\"\"\n",
    "    prompt_imagen = (\n",
    "        f\"Create a high-quality 3D-rendered image of a delicious homemade dish titled '{titulo_receta}'. \"\n",
    "        \"The image should look like professional editorial food photography: well-presented on a rustic table, \"\n",
    "        \"with soft natural lighting, shallow depth of field, and a clean, warm background. The dish should look fresh, realistic, and inviting.\"\n",
    "    )\n",
    "    try:\n",
    "        response = client.models.generate_content(\n",
    "            model=\"gemini-2.0-flash-preview-image-generation\", # Modelo de Google para generación de imágenes\n",
    "            contents=[prompt_imagen],\n",
    "            config=types.GenerateContentConfig(\n",
    "                response_modalities=['TEXT', 'IMAGE'] # Solicita tanto texto como imagen en la respuesta\n",
    "            )\n",
    "        )\n",
    "        imagen_generada_y_guardada = False\n",
    "        for part in response.candidates[0].content.parts:\n",
    "            if part.text is not None:\n",
    "                print(f\"📝 Descripción extra del modelo para la imagen:\\n{part.text}\")\n",
    "            elif part.inline_data is not None:\n",
    "                # Procesa los datos binarios de la imagen\n",
    "                image_data = part.inline_data.data\n",
    "                image = Image.open(BytesIO(image_data))\n",
    "                # Limpia el título para usarlo como un nombre de archivo válido\n",
    "                file_name_clean = \"\".join(c for c in titulo_receta if c.isalnum() or c in (' ', '_', '-')).lower().replace(' ', '_')\n",
    "                final_file_name = f\"promptchef_imagen_{file_name_clean}.png\"\n",
    "                image.save(final_file_name)\n",
    "                print(f\"🖼️ Imagen guardada como {final_file_name}\")\n",
    "                image.show() # Abre la imagen generada (requiere la librería Pillow/PIL)\n",
    "                imagen_generada_y_guardada = True\n",
    "\n",
    "        if not imagen_generada_y_guardada:\n",
    "            print(\"No se pudo obtener la imagen. La respuesta no contiene datos de imagen esperados.\")\n",
    "\n",
    "    except Exception as e:\n",
    "        print(f\"Error al generar la imagen: {e}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---",
    "## ⚙️ 5. Configuración y Ejecución Principal",
    "",
    "En esta sección es donde controlas cómo se generará tu receta. **Solo tienes que ajustar los valores aquí** y ejecutar la celda.",
    "",
    "### 🤔 Cómo elegir el modo de generación:",
    "",
    "* **Opción 1: Por Ingredientes:** Descomenta las líneas para `ingredientes_para_receta` y ajusta tus ingredientes. Asegúrate de que `plato_deseado_para_receta` esté vacío (`\"\"`).",
    "* **Opción 2: Por Nombre de Plato:** Descomenta las líneas para `plato_deseado_para_receta` y escribe el nombre del plato. Asegúrate de que `ingredientes_para_receta` esté vacío (`\"\"`).",
    "",
    "¡Solo una de las dos opciones debe tener un valor!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- TU SELECCIÓN ACTUAL ---",
    "# Modifica estas variables según lo que desees generar.\n",
    "# Asegúrate de dejar vacía la opción que NO uses.\n",
    "\n",
    "# Ejemplo para generar por INGREDIENTES:\n",
    "ingredientes_para_receta = \"papa, huevo, panceta, queso\"\n",
    "plato_deseado_para_receta = \"\" # ¡Deja esta vacía si usas ingredientes!\n",
    "\n",
    "# Ejemplo para generar por NOMBRE DE PLATO (descomenta si quieres usar esta opción y comenta la de arriba):\n",
    "# ingredientes_para_receta = \"\" \n",
    "# plato_deseado_para_receta = \"Lasagna de Carne\"\n",
    "\n",
    "# --- Configuración Opcional: Preferencias y Dificultad ---\n",
    "# Estas configuraciones se aplican a la generación de recetas en ambos escenarios.\n",
    "preferencias_receta = \"\" # Ejemplos: \"sin gluten\", \"vegetariana\", \"baja en carbohidratos\"\n",
    "dificultad_receta = \"intermedio\" # Opciones válidas: \"fácil\", \"intermedia\", \"avanzada\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 🚀 Lógica de Ejecución",
    "",
    "Esta celda contiene la lógica principal que decide qué función de generación de receta llamar (por ingredientes o por plato) y luego procede a mostrar la receta generada y a generar su imagen."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# --- Lógica Interna de Generación ---\n",
    "receta_generada = \"\"\n",
    "titulo_receta = \"\"\n",
    "tipo_de_generacion = \"\"\n",
    "\n",
    "if plato_deseado_para_receta:\n",
    "    print(f\"🧑‍🍳 Generando receta para: '{plato_deseado_para_receta}'...\")\n",
    "    receta_generada, titulo_receta = generar_receta_por_plato(\n",
    "        plato_deseado_para_receta, preferencias_receta, dificultad_receta\n",
    "    )\n",
    "    tipo_de_generacion = \"por nombre de plato\"\n",
    "elif ingredientes_para_receta:\n",
    "    print(\"🧑‍🍳 Generando receta a partir de ingredientes...\")\n",
    "    receta_generada, titulo_receta = generar_receta_por_ingredientes(\n",
    "        ingredientes_para_receta, preferencias_receta, dificultad_receta\n",
    "    )\n",
    "    tipo_de_generacion = \"por ingredientes\"\n",
    "else:\n",
    "    print(\"🤔 Por favor, especifica ingredientes o un nombre de plato para generar una receta.\")\n",
    "\n",
    "# --- Mostrar Receta y Generar Imagen ---\n",
    "# Este paso solo se ejecuta si se generó una receta con éxito.\n",
    "if tipo_de_generacion:\n",
    "    if \"Error\" not in titulo_receta:\n",
    "        print(f\"\\n📄 RECETA GENERADA ({tipo_de_generacion}):\\n\")\n",
    "        print(receta_generada)\n",
    "\n",
    "        print(\"\\n🖼️ Generando imagen del plato...\\n\")\n",
    "        generar_imagen_desde_titulo(titulo_receta)\n",
    "    else:\n",
    "        print(f\"❌ No se pudo proceder con la generación de la imagen debido a un error en la receta ({tipo_de_generacion}).\")\n",
    "\n",
    "print(\"\\n--- Fin de la generación de recetas ---\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}